In [1]:
'''
import requests

params = {
  'access_key': 'a061387a0536804d9d063ad007cc5d63'
}

api_result = requests.get('https://api.aviationstack.com/v1/flights', params)

api_response = api_result.json()

for flight in api_response['results']:
    if (flight['live']['is_ground'] is False):
        print(u'%s flight %s from %s (%s) to %s (%s) is in the air.' % (
            flight['airline']['name'],
            flight['flight']['iata'],
            flight['departure']['airport'],
            flight['departure']['iata'],
            flight['arrival']['airport'],
            flight['arrival']['iata']))
'''

"\nimport requests\n\nparams = {\n  'access_key': 'a061387a0536804d9d063ad007cc5d63'\n}\n\napi_result = requests.get('https://api.aviationstack.com/v1/flights', params)\n\napi_response = api_result.json()\n\nfor flight in api_response['results']:\n    if (flight['live']['is_ground'] is False):\n        print(u'%s flight %s from %s (%s) to %s (%s) is in the air.' % (\n            flight['airline']['name'],\n            flight['flight']['iata'],\n            flight['departure']['airport'],\n            flight['departure']['iata'],\n            flight['arrival']['airport'],\n            flight['arrival']['iata']))\n"

In [1]:
import requests
import pandas as pd
import sqlite3

In [2]:
params = {
    'access_key': 'a061387a0536804d9d063ad007cc5d63',
    'limit':30
}

api_response = {'pagination':{'limit':0}} # formatting for below code

In [3]:
# free api has connection errors sometimes

while api_response['pagination']['limit'] != params['limit']:
    try:
        api_result = requests.get('http://api.aviationstack.com/v1/flights', params)
        api_response = api_result.json()
    except:
        pass

In [14]:
# export the unaltered file

import json
with open('airplane.json', 'w') as fp:
    json.dump(api_response, fp)

In [18]:
total_data = api_response['data']
total_data[0]

{'flight_date': '2023-05-09',
 'flight_status': 'scheduled',
 'departure': {'airport': 'Chongqing Jiangbei International',
  'timezone': 'Asia/Shanghai',
  'iata': 'CKG',
  'icao': 'ZUCK',
  'terminal': 'T2',
  'gate': None,
  'delay': None,
  'scheduled': '2023-05-09T08:00:00+00:00',
  'estimated': '2023-05-09T08:00:00+00:00',
  'actual': None,
  'estimated_runway': None,
  'actual_runway': None},
 'arrival': {'airport': 'Fuzhou',
  'timezone': 'Asia/Shanghai',
  'iata': 'FOC',
  'icao': 'ZSFZ',
  'terminal': '1',
  'gate': None,
  'baggage': None,
  'delay': None,
  'scheduled': '2023-05-09T10:10:00+00:00',
  'estimated': '2023-05-09T10:10:00+00:00',
  'actual': None,
  'estimated_runway': None,
  'actual_runway': None},
 'airline': {'name': 'Sichuan Airlines', 'iata': '3U', 'icao': 'CSC'},
 'flight': {'number': '8103',
  'iata': '3U8103',
  'icao': 'CSC8103',
  'codeshared': None},
 'aircraft': None,
 'live': None}

In [19]:
# get keys for the new data frame
new_keys = total_data[0]['departure'].keys()
new_keys
# same for the arrival and departure


dict_keys(['airport', 'timezone', 'iata', 'icao', 'terminal', 'gate', 'delay', 'scheduled', 'estimated', 'actual', 'estimated_runway', 'actual_runway'])

In [20]:
# create dataframes to load data into 
arrival_df = pd.DataFrame(columns = new_keys)
departure_df = pd.DataFrame(columns = new_keys)

In [21]:
departure_df # now have data frames I can add the info to

,airport,timezone,iata,icao,terminal,gate,delay,scheduled,estimated,actual,estimated_runway,actual_runway


In [10]:
# can iterate through the items in the list, each corresponds to a flight, 
# and then we can add the data to the corresponding dataframes


In [22]:
# created two 
    
def update_departure(data,keys,df):
    for plane in data:
        dept_data = plane['departure']
        dept_temp = pd.DataFrame(dept_data, columns=keys, index=[0])
        df = pd.concat([df,dept_temp], ignore_index=True)
    return df

def update_arrival(data,keys,df):
    for plane in data:
        arr_data = plane['arrival']
        arr_temp = pd.DataFrame(arr_data, columns=keys, index=[0])
        df = pd.concat([df,arr_temp], ignore_index=True)
    return df
        
        

In [23]:
departure_df = update_departure(total_data, new_keys, departure_df)

In [24]:
departure_df.head()

,airport,timezone,iata,icao,terminal,gate,delay,scheduled,estimated,actual,estimated_runway,actual_runway
0,Chongqing Jiangbei International,Asia/Shanghai,CKG,ZUCK,T2,NaN,NaN,2023-05-09T08:00:00+00:00,2023-05-09T08:00:00+00:00,NaN,NaN,NaN
1,Chongqing Jiangbei International,Asia/Shanghai,CKG,ZUCK,T2,NaN,NaN,2023-05-09T08:00:00+00:00,2023-05-09T08:00:00+00:00,NaN,NaN,NaN
2,Chongqing Jiangbei International,Asia/Shanghai,CKG,ZUCK,T2,NaN,NaN,2023-05-09T08:00:00+00:00,2023-05-09T08:00:00+00:00,NaN,NaN,NaN
3,Polonia,Asia/Jakarta,KNO,WIMM,NaN,NaN,NaN,2023-05-09T07:01:00+00:00,2023-05-09T07:01:00+00:00,NaN,NaN,NaN
4,Polonia,Asia/Jakarta,KNO,WIMM,NaN,NaN,NaN,2023-05-09T07:00:00+00:00,2023-05-09T07:00:00+00:00,NaN,NaN,NaN


In [25]:
arrival_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   airport           0 non-null      object
 1   timezone          0 non-null      object
 2   iata              0 non-null      object
 3   icao              0 non-null      object
 4   terminal          0 non-null      object
 5   gate              0 non-null      object
 6   delay             0 non-null      object
 7   scheduled         0 non-null      object
 8   estimated         0 non-null      object
 9   actual            0 non-null      object
 10  estimated_runway  0 non-null      object
 11  actual_runway     0 non-null      object
dtypes: object(12)
memory usage: 0.0+ bytes


In [26]:
arrival_df = update_arrival(total_data,new_keys,arrival_df)

In [27]:
arrival_df.head()

,airport,timezone,iata,icao,terminal,gate,delay,scheduled,estimated,actual,estimated_runway,actual_runway
0,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,NaN,2023-05-09T10:10:00+00:00,2023-05-09T10:10:00+00:00,NaN,NaN,NaN
1,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,NaN,2023-05-09T10:10:00+00:00,2023-05-09T10:10:00+00:00,NaN,NaN,NaN
2,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,NaN,2023-05-09T10:10:00+00:00,2023-05-09T10:10:00+00:00,NaN,NaN,NaN
3,Binaka,Asia/Jakarta,GNS,WIMB,NaN,NaN,NaN,2023-05-09T08:00:00+00:00,2023-05-09T08:00:00+00:00,NaN,NaN,NaN
4,Sultan Syarif Kasim Ii,Asia/Jakarta,PKU,WIBB,NaN,NaN,NaN,2023-05-09T08:25:00+00:00,2023-05-09T08:25:00+00:00,NaN,NaN,NaN


In [28]:
arrival_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   airport           30 non-null     object
 1   timezone          30 non-null     object
 2   iata              30 non-null     object
 3   icao              30 non-null     object
 4   terminal          12 non-null     object
 5   gate              1 non-null      object
 6   delay             0 non-null      object
 7   scheduled         30 non-null     object
 8   estimated         30 non-null     object
 9   actual            0 non-null      object
 10  estimated_runway  0 non-null      object
 11  actual_runway     0 non-null      object
dtypes: object(12)
memory usage: 2.9+ KB


In [29]:
# before we do any transformations, we will make a copy of the dataframes 

departure_copy = departure_df.copy(deep=True)
arrival_copy = arrival_df.copy(deep=True)

In [30]:
depature_df = departure_copy
arrival_df = arrival_copy

In [31]:
# now what do I want to do to these dataframes?

# 1: change dates to time and date in different columns x 
# 2: change 
# 3: remove NaN values x 
# 4: use sql to pull ? data

# how to use sql

In [32]:
arrival_df.columns

Index(['airport', 'timezone', 'iata', 'icao', 'terminal', 'gate', 'delay',
       'scheduled', 'estimated', 'actual', 'estimated_runway',
       'actual_runway'],
      dtype='object')

In [33]:
# 1: drop all the null columns with no information

departure_df = departure_df.dropna(axis=1,how='all')
arrival_df = arrival_df.dropna(axis=1,how='all')

arrival_df.head()
# we only have the airport, timezone, iata & icao codes, termial, gate, and scheduled/estimated times left

,airport,timezone,iata,icao,terminal,gate,scheduled,estimated
0,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,2023-05-09T10:10:00+00:00,2023-05-09T10:10:00+00:00
1,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,2023-05-09T10:10:00+00:00,2023-05-09T10:10:00+00:00
2,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,2023-05-09T10:10:00+00:00,2023-05-09T10:10:00+00:00
3,Binaka,Asia/Jakarta,GNS,WIMB,NaN,NaN,2023-05-09T08:00:00+00:00,2023-05-09T08:00:00+00:00
4,Sultan Syarif Kasim Ii,Asia/Jakarta,PKU,WIBB,NaN,NaN,2023-05-09T08:25:00+00:00,2023-05-09T08:25:00+00:00


In [34]:
# 2: change date/times

# scheduled and estiamted: turn into two different columns each
# the date (dd-mm-yyy) and time (hh:mm:ss) get rid of anything smaller than a second


In [35]:
'''
ISO 8601 represents date and time by starting with the year, 
followed by the month, the day, the hour, the minutes, seconds and milliseconds. 
For example, 2020-07-10 15:00:00.000, represents the 10th of July 2020 at 3 p.m. 
(in local time as there is no time zone offset specified—more on that below)
'''

'\nISO 8601 represents date and time by starting with the year, \nfollowed by the month, the day, the hour, the minutes, seconds and milliseconds. \nFor example, 2020-07-10 15:00:00.000, represents the 10th of July 2020 at 3 p.m. \n(in local time as there is no time zone offset specified—more on that below)\n'

In [36]:
# create two new columns containing the time and date of the scheduled/estimated arrival/departure

# split the shecduled and estimated columns into sheduled_date, scheduled_time, estimated_date, estimated_time
departure_df[['scheduled_date', 'scheduled_time']] = departure_df["scheduled"].apply(lambda x: pd.Series(str(x).split("T")))
departure_df[['estimated_date', 'estimated_time']] = departure_df["estimated"].apply(lambda x: pd.Series(str(x).split("T")))

arrival_df[['scheduled_date', 'scheduled_time']] = arrival_df["scheduled"].apply(lambda x: pd.Series(str(x).split("T")))
arrival_df[['estimated_date', 'estimated_time']] = arrival_df["estimated"].apply(lambda x: pd.Series(str(x).split("T")))

<ipython-input-36-92773c2a0bc2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arrival_df[['scheduled_date', 'scheduled_time']] = arrival_df["scheduled"].apply(lambda x: pd.Series(str(x).split("T")))
<ipython-input-36-92773c2a0bc2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arrival_df[['scheduled_date', 'scheduled_time']] = arrival_df["scheduled"].apply(lambda x: pd.Series(str(x).split("T")))
<ipython-input-36-92773c2a0bc2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arrival_df[['scheduled_date', 'scheduled_time']] = arrival_df["scheduled"].apply(lambda x: pd.Series(str(x).split("T")))
<ipython-input-36-92773c2a0bc2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [37]:
# drop the miliseconds
departure_df['scheduled_time'] = departure_df['scheduled_time'].apply(lambda x: str(x)[:8])
departure_df['estimated_time'] = departure_df['estimated_time'].apply(lambda x: str(x)[:8])

arrival_df['scheduled_time'] = arrival_df['scheduled_time'].apply(lambda x: str(x)[:8])
arrival_df['estimated_time'] = arrival_df['estimated_time'].apply(lambda x: str(x)[:8])



In [38]:
# drop the old columns we no longer need

departure_df = departure_df.drop(['scheduled','estimated'], axis=1)
arrival_df = arrival_df.drop(['scheduled','estimated'], axis=1)

In [39]:
arrival_df.head()

,airport,timezone,iata,icao,terminal,gate,scheduled_date,scheduled_time,estimated_date,estimated_time
0,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,2023-05-09,10:10:00,2023-05-09,10:10:00
1,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,2023-05-09,10:10:00,2023-05-09,10:10:00
2,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,2023-05-09,10:10:00,2023-05-09,10:10:00
3,Binaka,Asia/Jakarta,GNS,WIMB,NaN,NaN,2023-05-09,08:00:00,2023-05-09,08:00:00
4,Sultan Syarif Kasim Ii,Asia/Jakarta,PKU,WIBB,NaN,NaN,2023-05-09,08:25:00,2023-05-09,08:25:00


In [41]:
# transform the new column's values into datetime objects rather than strings

# times
departure_df['scheduled_time'] = pd.to_datetime(departure_df['scheduled_time']).dt.time
departure_df['estimated_time'] = pd.to_datetime(departure_df['estimated_time']).dt.time
arrival_df['scheduled_time'] = pd.to_datetime(arrival_df['scheduled_time']).dt.time
arrival_df['estimated_time'] = pd.to_datetime(arrival_df['estimated_time']).dt.time

# dates
departure_df['scheduled_date'] = pd.to_datetime(departure_df['scheduled_date']).dt.date
departure_df['estimated_date'] = pd.to_datetime(departure_df['estimated_date']).dt.date
arrival_df['scheduled_date'] = pd.to_datetime(arrival_df['scheduled_date']).dt.date
arrival_df['estimated_date'] = pd.to_datetime(arrival_df['estimated_date']).dt.date

TypeError: <class 'datetime.time'> is not convertible to datetime

In [49]:
arrival_df.head()

,airport,timezone,iata,icao,terminal,gate,scheduled_date,scheduled_time,estimated_date,estimated_time
0,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,2023-05-09,10:10:00,2023-05-09,10:10:00
1,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,2023-05-09,10:10:00,2023-05-09,10:10:00
2,Fuzhou,Asia/Shanghai,FOC,ZSFZ,1,NaN,2023-05-09,10:10:00,2023-05-09,10:10:00
3,Binaka,Asia/Jakarta,GNS,WIMB,NaN,NaN,2023-05-09,08:00:00,2023-05-09,08:00:00
4,Sultan Syarif Kasim Ii,Asia/Jakarta,PKU,WIBB,NaN,NaN,2023-05-09,08:25:00,2023-05-09,08:25:00


In [43]:
departure_df.head()

,airport,timezone,iata,icao,terminal,gate,delay,scheduled_date,scheduled_time,estimated_date,estimated_time
0,Chongqing Jiangbei International,Asia/Shanghai,CKG,ZUCK,T2,NaN,NaN,2023-05-09,08:00:00,2023-05-09,08:00:00
1,Chongqing Jiangbei International,Asia/Shanghai,CKG,ZUCK,T2,NaN,NaN,2023-05-09,08:00:00,2023-05-09,08:00:00
2,Chongqing Jiangbei International,Asia/Shanghai,CKG,ZUCK,T2,NaN,NaN,2023-05-09,08:00:00,2023-05-09,08:00:00
3,Polonia,Asia/Jakarta,KNO,WIMM,NaN,NaN,NaN,2023-05-09,07:01:00,2023-05-09,07:01:00
4,Polonia,Asia/Jakarta,KNO,WIMM,NaN,NaN,NaN,2023-05-09,07:00:00,2023-05-09,07:00:00


In [44]:
# 'Loading': export both files now that transformatin is done

arrival_df.to_csv('arrival.csv',index=False)
departure_df.to_csv('departure.csv',index=False)


In [ ]:
# can just load datasets now and create the sql tables to work on them,
# dont' need to run request again

# for final, I would try to get somthign like 100 to 500 entries

In [7]:
import pandas as pd
import sqlite3
departure_df = pd.read_csv("departure.csv",header=0)
arrival_df = pd.read_csv('arrival.csv',header=0)

In [45]:
# there's nothing more I really want to do to these datasets
# lets use sql to make some queries on some things we would like to know

In [9]:
con = sqlite3.connect("airplane.db")
cur = con.cursor()

In [29]:
departure_df.to_sql("departure",con,if_exists='replace',index=False)
arrival_df.to_sql('arrival',con,if_exists='replace',index=False)

OperationalError: table "departure" already exists

In [50]:
# change time zone from continent/city to two different columns
# pull specific terminal/continent entries
# pull flight from certain times
# done - work on annother project

In [62]:
# split the timezone into continent and city

# departure
# create city column and take substring from timezone
# rename timezone to continent
# change continent values to substring, to only contain continent
cur.execute("select timezone trim(substr(timezone, instr(timezone,'/')+1)) as city from departure")

#cur.execute('alter table departure rename column timezone to continent')

#cur.execute("update departure set continent = substr(continent, 1, instr(continent,'/')-1)")



OperationalError: near "=": syntax error

In [60]:
# do same thing for arrival
cur.execute("select timezone, trim(substr(timezone, instr(timezone,'/')+1)) as city from arrival")

cur.execute('alter table arrival rename column timezone to continent')

cur.execute("update arrival set continent = substr(continent, 1, instr(continent,'/')-1)")

OperationalError: no such column: timezone

In [61]:
cur.execute("""select city from departure""").fetchall()

OperationalError: no such column: city

In [25]:
# drop the original column
# sqlite has limited functionality when it comes to dropping and adding columns

cur.execute("""CREATE TABL t1_backup AS SELECT a, b FROM t1;
DROP TABLE t1;
ALTER TABLE t1_backup RENAME TO t1""")

cur.execute("""create table dept_backup as select""")


OperationalError: near ",": syntax error

In [26]:
cur.execute("""create view ViewName as
select col1, col2, col3, .... 
    FROM tableName""")

OperationalError: near "drop": syntax error

In [17]:
# pull the two columns 


[('Chongqing Jiangbei International', 'Asia/Shanghai', 'CKG', 'ZUCK', 'T2', None, None, '2023-05-09', '08:00:00', '2023-05-09', '08:00:00'), ('Chongqing Jiangbei International', 'Asia/Shanghai', 'CKG', 'ZUCK', 'T2', None, None, '2023-05-09', '08:00:00', '2023-05-09', '08:00:00'), ('Chongqing Jiangbei International', 'Asia/Shanghai', 'CKG', 'ZUCK', 'T2', None, None, '2023-05-09', '08:00:00', '2023-05-09', '08:00:00'), ('Polonia', 'Asia/Jakarta', 'KNO', 'WIMM', None, None, None, '2023-05-09', '07:01:00', '2023-05-09', '07:01:00'), ('Polonia', 'Asia/Jakarta', 'KNO', 'WIMM', None, None, None, '2023-05-09', '07:00:00', '2023-05-09', '07:00:00'), ('Polonia', 'Asia/Jakarta', 'KNO', 'WIMM', None, None, None, '2023-05-09', '06:30:00', '2023-05-09', '06:30:00'), ('Polonia', 'Asia/Jakarta', 'KNO', 'WIMM', None, None, None, '2023-05-09', '06:00:00', '2023-05-09', '06:00:00'), ('Polonia', 'Asia/Jakarta', 'KNO', 'WIMM', None, None, None, '2023-05-09', '05:40:00', '2023-05-09', '05:40:00'), ('Polonia